***importing packages and libraries***

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import RandomBrightness, RandomContrast, RandomZoom
from configs import modelConfigs
import pathlib as pl
import matplotlib.pyplot as plt

In [ ]:
def pre_process_image(image_path, target_height):
    # Load image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=1)  # Decode to grayscale
    
    # Convert image to float32 and normalize to [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)

    shape = tf.shape(image)

    # Convert to numpy array
    h, w, _ = shape.numpy()

    # Calculate new width based on target height
    aspect_ratio = tf.cast(w, tf.float32) / tf.cast(h, tf.float32)
    new_width = tf.cast(target_height * aspect_ratio, tf.int32)

    # Resize image
    image_resized = tf.image.resize(image, [target_height, new_width])

    return image_resized

In [ ]:
def build_augmentation_model():
    augmentation_model = tf.keras.Sequential([
        RandomBrightness(factor=(-0.1, 0.1), value_range=(0, 1)),
        RandomContrast(factor=0.05),
        RandomZoom(height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2), fill_mode='constant', fill_value=1.0)     
    ])
    return augmentation_model


In [ ]:
#function for inspiration not for actual usage
"""
def input_batch_generator(image_paths, target_height, target_width, batch_size, augmentation_probability=0.2):
    while True:
        batch_images = []
        for image_path in image_paths:
            image = pre_process_image(image_path, target_height)
            if np.random.rand() < augmentation_probability:
                image = augment_image(image)
            if image.shape[1] < target_width:
                pad_width = target_width - image.shape[1]
                image = np.pad(image, ((0, 0), (0, pad_width)), mode='constant', constant_values=1.0)
            else:
                image = image[:, :target_width]
            batch_images.append(image)
            if len(batch_images) == batch_size:
                yield np.array(batch_images)
                batch_images = []
"""


In [ ]:
# build the 2 functions for retrive the necessary image file path for each folder
"""
brainstorm:
batch generator...
batch_generator loops through file paths and feeds that file path into a function that outputs
a tuple with an image path and its associated XML path.

segmenting the loops...
have 2 seperate loops for each folder and varaibles to keep track of the 
final image that was passed into the training batch (to remember the final image placed in the batch).

input and output retrieval for batch generation...


"""


In [ ]:
def batch_generator(image_paths , batch_size, target_height, augmentation_probability = 0.3, batch_data_split = (0.4, 0.6)):
    forms_ex = pl.Path(image_paths[0])
    sentence_ex = pl.Path(image_paths[1])
    #keep track of last image from the imageg data base 
    last_image_forms = 0
    last_image_sentence = 0

    forms_tot_num = round(batch_size * batch_data_split[0])
    senetence_tot_num = round(batch_size * batch_data_split[1])
    
    augmentation_model = build_augmentation_model()
    while True:
        batch = [] #initilaise batch array
        
        pos_counter = 0
        for image_path in forms_ex.iterdir(): #loop thorugh the forms array
            # split the batch by the percentage outlined
            if pos_counter == last_image_forms + forms_tot_num + 1:
                #number of forms added
                break
            elif pos_counter >= last_image_forms:
                pos_counter += 1
                #change the imagepath data structure from pathlib structures into string for image preporcessing
                if not isinstance(image_path,str):
                    image_path = str(image_path)
                image = pre_process_image(image_path, target_height)
                #apply augementation based on probability
                if np.random.rand() < augmentation_probability:
                    image = augmentation_model(image)
                batch.append(image)
        break
    #convert batch array into numpy array to allow for model training  
    return np.array(batch)


In [ ]:
# A function written to display a batch
#I used it to ensure batch generator was working alongside the augmentor model

def inpute_batch_displayer(batch):

    num_images = len(batch)
    cols = 5  # Adjust number of columns to fit your needs
    rows = (num_images + cols - 1) // cols

    fig, axes = plt.subplots(rows, cols, figsize=(15, rows * 3))
    axes = axes.flatten()  # Flatten to easily index axes

    for i in range(num_images):
        ax = axes[i]
        ax.imshow(batch[i], cmap='gray')
        ax.axis('off')
        # Add index label on top of each image
        ax.text(0.5, 1.05, str(i), ha='center', va='center', transform=ax.transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=0.7))

    # Hide any empty subplots
    for i in range(num_images, len(axes)):
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
c = modelConfigs()

batch = batch_generator(c.image_paths, c.test_batch_size, c.image_height, c.augmentation_probability, c.data_split)
print(c.test_batch_size)
print(batch)

In [ ]:
inpute_batch_displayer(batch)